# Data exploration of grand Lyon open data (Silos de Verre)
Data from https://data.grandlyon.com/environnement/silo-verre-de-la-mftropole-de-lyon/

In [67]:
import pandas as pd
import json
import numpy as np
import requests
import os

In [42]:
DOWNLOAD_LINK = 'https://download.data.grandlyon.com/wfs/grandlyon?SERVICE=WFS&VERSION=2.0.0&outputformat=GEOJSON&maxfeatures=-1&request=GetFeature&typename=gic_collecte.gicsiloverre&SRSNAME=urn:ogc:def:crs:EPSG::4171'
resp = requests.get(DOWNLOAD_LINK)
silos_json = json.loads(resp.text)

In [43]:
len(silos_json['features'])

2662

In [46]:
silos = pd.DataFrame(silos_json['features'])
silos

,geometry,properties,type
0,"{'coordinates': [4.77591697654058, 45.81498325...","{'voie': 'Route Départementale 306', 'identifi...",Feature
1,"{'coordinates': [4.832169644313046, 45.8651308...","{'voie': 'Rue Jean Chirat', 'identifiant': '10...",Feature
2,"{'coordinates': [4.761649996466955, 45.7375483...","{'voie': 'Rue De L'Eglise', 'identifiant': '51...",Feature
3,"{'coordinates': [4.899202034366342, 45.8223854...","{'voie': 'Rue de la République', 'identifiant'...",Feature
4,"{'coordinates': [4.797909237184583, 45.7540592...","{'voie': 'Rue Docteur Albéric Pont', 'identifi...",Feature
5,"{'coordinates': [4.826300724766074, 45.7596177...","{'voie': 'Rue du Doyenné', 'identifiant': '698...",Feature
6,"{'coordinates': [4.761198454792476, 45.7504297...","{'voie': 'Chemin de la Poterie', 'identifiant'...",Feature
7,"{'coordinates': [4.896801131180903, 45.7556119...","{'voie': 'Rue Paul Kruger / Rue de Wett', 'ide...",Feature
8,"{'coordinates': [4.845475107021888, 45.7714697...",{'voie': 'Place Puvis de Chavannes/Rue de Créq...,Feature
9,"{'coordinates': [4.847153034557991, 45.7746882...",{'voie': 'Rue Félix Jacquier angle Rue Bareme'...,Feature


In [56]:
geos = pd.DataFrame([{'coordinates': v['coordinates']} for v in silos.geometry.values])
geos

,coordinates
0,"[4.77591697654058, 45.81498325877725]"
1,"[4.832169644313046, 45.86513080953407]"
2,"[4.761649996466955, 45.737548399421755]"
3,"[4.899202034366342, 45.82238548042423]"
4,"[4.797909237184583, 45.754059271313935]"
5,"[4.826300724766074, 45.759617726690514]"
6,"[4.761198454792476, 45.75042974389414]"
7,"[4.896801131180903, 45.75561190230803]"
8,"[4.845475107021888, 45.77146971000744]"
9,"[4.847153034557991, 45.774688220625954]"


In [63]:
props = pd.DataFrame(silos.properties.values.tolist())
props

,code_insee,code_postal,commune,gestionnaire,gid,identifiant,numerodansvoie,observation,voie
0,69116,69760,LIMONEST,Col Nord-Ouest,1,1010,1244,Arrêt de bus vers Aviva,Route Départementale 306
1,69003,69250,ALBIGNY-SUR-SAONE,Col Nord-Ouest,2,1004,1,,Rue Jean Chirat
2,69089,69340,FRANCHEVILLE,Col Sud,3,5175,,,Rue De L'Eglise
3,69286,69140,RILLIEUX-LA-PAPE,Col Nord-Ouest,4,4350,8,,Rue de la République
4,69385,69005,LYON 5 ÈME,Col Nord-Ouest,5,1511,,angle rue des Granges,Rue Docteur Albéric Pont
5,69385,69005,LYON 5 ÈME,Col Nord-Ouest,6,6986,7,,Rue du Doyenné
6,69089,69340,FRANCHEVILLE,Col Sud,7,6962,Face au N°,"Face au n° 1, après les stationnements",Chemin de la Poterie
7,69266,69100,VILLEURBANNE,Col Est,8,3759,,,Rue Paul Kruger / Rue de Wett
8,69386,69006,LYON 6 ÈME,Col Est,9,2004,,DEUX SILOS,Place Puvis de Chavannes/Rue de Créqui
9,69386,69006,LYON 6 ÈME,Col Est,10,2044,,,Rue Félix Jacquier angle Rue Bareme


In [68]:
np.unique(props.gestionnaire)

array(['Col Est', 'Col Nord-Ouest', 'Col Sud'], dtype=object)

In [70]:
np.unique(props.commune)

array(['ALBIGNY-SUR-SAONE', 'BRON', 'CAILLOUX-SUR-FONTAINES',
       'CALUIRE-ET-CUIRE', "CHAMPAGNE-AU-MONT-D'OR",
       'CHARBONNIERES-LES-BAINS', 'CHARLY', 'CHASSIEU',
       "COLLONGES-AU-MONT-D'OR", 'CORBAS', "COUZON-AU-MONT-D'OR",
       'CRAPONNE', "CURIS-AU-MONT-D'OR", 'DARDILLY', 'DECINES-CHARPIEU',
       'ECULLY', 'FEYZIN', 'FLEURIEU-SUR-SAONE', 'FONTAINES-SAINT-MARTIN',
       'FONTAINES-SUR-SAONE', 'FRANCHEVILLE', 'GENAY', 'GIVORS', 'GRIGNY',
       'IRIGNY', 'JONAGE', 'LA MULATIERE', 'LA TOUR-DE-SALVAGNY',
       'LIMONEST', 'LISSIEU', 'LYON 1 ER', 'LYON 2 ÈME', 'LYON 3 ÈME',
       'LYON 4 ÈME', 'LYON 5 ÈME', 'LYON 6 ÈME', 'LYON 7 ÈME',
       'LYON 8 ÈME', 'LYON 9 ÈME', "MARCY-L'ETOILE", 'MEYZIEU', 'MIONS',
       'MONTANAY', 'NEUVILLE-SUR-SAONE', 'OULLINS', 'PIERRE-BENITE',
       "POLEYMIEUX-AU-MONT-D'OR", 'QUINCIEUX', 'RILLIEUX-LA-PAPE',
       'ROCHETAILLEE-SUR-SAONE', "SAINT-CYR-AU-MONT-D'OR",
       "SAINT-DIDIER-AU-MONT-D'OR", 'SAINT-FONS', 'SAINT-GENIS-LAVAL',
 

In [86]:
[p for _, p in props.iterrows() if 'sonde' in p.observation or 'capteur' in p.observation or 'remplissage' in p.observation]

[code_insee                                    69383
 code_postal                                   69003
 commune                                  LYON 3 ÈME
 gestionnaire                                Col Est
 gid                                             989
 identifiant                                    2753
 numerodansvoie                                     
 observation       Equipé d'une sonde de remplissage
 voie                   Rue André Philip / paul Bert
 Name: 988, dtype: object]